In [1]:
import gc
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os
from matplotlib import pyplot
#from google.colab import drive
#drive.mount('/content/gdrive')
import tensorflow as tf
from keras import backend as K

from keras.utils.np_utils import to_categorical
from keras.models import Model
from keras.applications import InceptionV3
from keras.layers import GlobalAveragePooling2D,Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import preprocess_input
from keras.callbacks import ModelCheckpoint



#from keras.optimizers import RMSprop
#from keras.layers.advanced_activations import LeakyReLU


Using TensorFlow backend.


'\nimport tensorflow as tf\nfrom keras import backend as K\nfrom keras.models import Model, Sequential\nfrom keras import regularizers, optimizers\nfrom keras.applications import InceptionV3\nfrom keras.applications.inception_v3 import preprocess_input\nfrom keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img\nfrom keras.layers import Dropout, Flatten, Dense\nfrom keras.layers import GlobalAveragePooling2D,Conv2D,MaxPooling2D,Activation\nfrom keras.utils.vis_utils import plot_model\nfrom keras.utils.np_utils import to_categorical\n\n'

In [2]:

num_cores = 1

config = tf.ConfigProto(intra_op_parallelism_threads=1,
                       inter_op_parallelism_threads=1,
                       allow_soft_placement=True,
                       device_count = {'GPU': 1}
                       )

#{'GPU': 1,  'CPU': 1}
session = tf.Session(config=config)
K.set_session(session)


"\nrepo_path = '/content/gdrive/My Drive/Course2/NDSC-advanced_competition'\ndoc_path = repo_path\n\nworking_area = ['fashion', 'beauty','mobile']\n\n"

In [3]:

doc_path = os.path.curdir
base_path = '/dataset/NDSC'
repo_path = os.path.join('e:',base_path)


working_area = ['fashion', 'beauty','mobile']
fixbug=False

def without_new(old):
    return old.replace("new_", "")
  
def is_class(old, classes):
    return old+'_'+classes
  
def strnumcombin(a):
    return a[0]+' '+a[1]

def data_prepare(working_area, fixbug=False):    
    df_train = {}
    df_val = {}
    df_profile = {}
    for work in working_area:
        df_train[work] = pd.read_csv(os.path.join(doc_path,work+"_data_info_train_competition.csv"))
    for work in working_area:
        df_val[work] = pd.read_csv(os.path.join(doc_path,work+"_data_info_val_competition.csv"))
    for work in working_area:
        df_profile[work] = pd.read_json(os.path.join(doc_path,work+"_profile_train.json"))
    
    for work in working_area:
        df_train[work].rename(columns=lambda x:x.replace(' ','_'), inplace=True)
        df_val[work].rename(columns=lambda x:x.replace(' ','_'), inplace=True)
        df_profile[work].rename(columns=lambda x:x.replace(' ','_'), inplace=True)
    
    # can only implement once, fix the path error with no .jpg
    for work in working_area:
        if work == 'fashion':
            if fixbug == False:
                new_image_path = (df_train['fashion']['image_path'][0:219702]+ '.jpg').tolist()+df_train['fashion']['image_path'][219702:].tolist()
                df_train['fashion']['image_path'] = new_image_path
                df_val['fashion']['image_path']=df_val['fashion'].image_path.apply(without_new)
                fixbug = True
  
    df_class={}
    for work in working_area:
        df_working_class={}
        sub_classes = df_train[work].columns.tolist()[3:]
        for classes in sub_classes:
            df_working_class[classes]=df_train[work][['image_path', classes]].copy()
            df_working_class[classes].dropna(axis=0, how='any',inplace=True)
        df_class[work]= df_working_class
      
    df_tag={}
    for work in working_area:
        df_working_tag={}
        sub_classes = df_train[work].columns.tolist()[3:]
        for classes in sub_classes:
            df_working_tag[classes]=df_profile[work][classes].value_counts().keys().tolist()
            df_working_tag[classes].sort()
        df_tag[work]= df_working_tag
    
    df_cat={}
    for work in working_area:
        df_working_cat={}
        sub_classes = df_train[work].columns.tolist()[3:] # get the classes for each work
        for classes in sub_classes:
            max_model = max(df_tag[work][classes]) + 1 # in profile json, how many tags for each classes
            df_working_cat[classes]=to_categorical(df_class[work][classes][classes],int(max_model))
        df_cat[work]= df_working_cat 
    
    for work in working_area:
        sub_classes = df_train[work].columns.tolist()[3:]
        for classes in sub_classes:
            for i in df_tag[work][classes]:
                df_class[work][classes][classes+'_'+str(i)] = df_cat[work][classes][:, int(i)]
            df_class[work][classes].drop(classes,axis=1, inplace=True)
   
    traing_classes={}
    for work in working_area:
        traing_classes_cat={}
        sub_classes = df_train[work].columns.tolist()[3:]
        for classes in sub_classes:
            traing_classes_cat_temp=df_class[work][classes].columns.tolist()
            traing_classes_cat[classes]=traing_classes_cat_temp[1:]
        traing_classes[work]= traing_classes_cat
    
    return df_train,df_val,df_class,traing_classes

In [4]:
def model_pre_adjust(num_classes=4, trainable=True):    
    base_model = InceptionV3(input_shape=(299, 299, 3), weights='imagenet', include_top=False)
    #base_model.summary()
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    #x = LeakyReLU(alpha=0.1)(x)
    #predictions = Dense(5, activation='softmax')(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    model_test1 = Model(inputs=base_model.input, outputs = predictions)
    if trainable:
        for layer in model_test1.layers[:140]:
            layer.trainable = False
        for layer in model_test1.layers[140:]:
            layer.trainable = True
    else:
        for layer in base_model.layers:
            layer.trainable = False 
    return model_test1

In [5]:
def data_pre_generator(work, classes):
    #from keras.preprocessing.image import ImageDataGenerator
    #from keras.applications.inception_v3 import preprocess_input
    sz=299
    batch_size = 128
    train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                       width_shift_range=0.2,
                                       height_shift_range=0.2,
                                       rotation_range=20,
                                       shear_range=0.2,
                                       zoom_range=0.2,
                                       horizontal_flip=True,
                                       validation_split = 0.15)
    test_datagen = ImageDataGenerator(rescale=1./255.)
    train_generator = train_datagen.flow_from_dataframe(dataframe=df_class[work][classes],
                                                        directory=repo_path, 
                                                        x_col = 'image_path',
                                                        y_col = traing_classes[work][classes],
                                                        class_mode = "other", 
                                                        subset="training",
                                                        target_size=(sz, sz),
                                                        batch_size=batch_size,
                                                        seed=42,
                                                        shuffle=True
                                                       )
    
    validation_generator = train_datagen.flow_from_dataframe(dataframe=df_class[work][classes],
                                                             directory=repo_path, 
                                                             x_col = 'image_path',
                                                             y_col = traing_classes[work][classes],
                                                             class_mode = "other", 
                                                             subset="validation",
                                                             target_size=(sz, sz),
                                                             batch_size=batch_size,
                                                             seed=42,
                                                             shuffle=True
                                                            )
    
    test_generator=test_datagen.flow_from_dataframe(dataframe=df_val[work],
                                                    directory=repo_path,
                                                    x_col='image_path',
                                                    y_col=None,
                                                    class_mode = None,
                                                    batch_size=batch_size,
                                                    seed=42,
                                                    shuffle=False,
                                                    target_size=(sz, sz))
    return train_generator, validation_generator,test_generator

In [6]:
def train_data(model_test1, train_generator, validation_generator, test_generator, work, classes):
    #from keras.callbacks import ModelCheckpoint
    STEP_SIZE_TRAIN = train_generator.n // train_generator.batch_size
    train_cut = train_generator.n%train_generator.batch_size
    if train_cut!=0:
        STEP_SIZE_TRAIN +=1
    STEP_SIZE_VALID = validation_generator.n // validation_generator.batch_size
    valid_cut = validation_generator.n%validation_generator.batch_size
    if valid_cut!=0:
        STEP_SIZE_VALID +=1
    STEP_SIZE_TEST = test_generator.n // test_generator.batch_size
    test_cut = test_generator.n%test_generator.batch_size
    if test_cut !=0:
        STEP_SIZE_TEST +=1
    model_test1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    filepath=doc_path +'/models/weights-best_'+work+classes+'.hdf5'
    checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    hist_test_classes = model_test1.fit_generator(generator = train_generator,
                                                  steps_per_epoch = STEP_SIZE_TRAIN,
                                                  validation_data = validation_generator,
                                                  validation_steps = STEP_SIZE_VALID,
                                                  epochs = 10
                                                 )
    model_test1.save(doc_path +'/models/clasif10_model1_'+work+classes+'.h5')
    evaluate_test_classes=model_test1.evaluate_generator(generator=validation_generator,
                                                         steps=STEP_SIZE_VALID
                                                        )
    print('evaluation accuracy for '+work+'_'+classes +'='+ str(evaluate_test_classes[1]))
    test_generator.reset()
    pred_test_classes=model_test1.predict_generator(test_generator,
                                                    steps=STEP_SIZE_TEST,
                                                    verbose=1
                                                   )
    del model_test1
    return hist_test_classes, evaluate_test_classes, pred_test_classes

In [7]:
def submit_data_prepare(pred_test):
    df_base = df_val[work][['itemid']].copy()
    df_base_classes=pd.DataFrame()
    df_base['itemid']=df_base.itemid.apply(str)
    df_base['itemid']=df_base.itemid.apply((lambda x: is_class(x, classes)))
    #tagging
    print(pred_test)
    predicted_indices=np.argsort((-1)*pred_test,axis=1)
    predicted_top=np.delete(predicted_indices, np.s_[2:], axis=1)
    #df_base['tagging']=predicted_top # 2D array ? 
    predicted_top=predicted_top.astype(str)
    predicted_top_object=np.array(list(predicted_top), dtype=np.object)
    predicted_top_series = pd.Series(list(predicted_top_object))
    predicted_top_series=predicted_top_series.apply(strnumcombin)
    df_base['tagging']=predicted_top_series
    df_base.rename(columns={'itemid':'id'}, inplace=True)
    df_base_classes=df_base_classes.append(df_base, ignore_index=True)
    
    return df_base_classes

In [ ]:
df_submit=pd.DataFrame()
hist_test={}
evaluate_test={} 
pred_test={} 

working_area = ['fashion', 'beauty','mobile']

df_train,df_val,df_class,traing_classes=data_prepare(working_area, fixbug)

for work in working_area:
    hist_test_classes={}
    evaluate_test_classes={}
    pred_test_classes={}
    df_base_classes=pd.DataFrame()
    sub_classes = df_train[work].columns.tolist()[3:]
    for classes in sub_classes:
        K.clear_session()
        df_base = df_val[work][['itemid']].copy()
        model_test1=model_pre_adjust(num_classes=len(traing_classes[work][classes]), trainable=True)
        train_generator, validation_generator,test_generator = data_pre_generator(work, classes)
        hist_test_classes[classes], evaluate_test_classes[classes],pred_test_classes[classes]=train_data(model_test1, train_generator, validation_generator, test_generator, work, classes)
        df_base_classes=df_base_classes.append(submit_data_prepare(pred_test_classes[classes]),ignore_index=True)
        df_base_classes.to_csv(doc_path +'/'+work+'_'+classes+'submit.csv')
    hist_test[work]=hist_test_classes
    evaluate_test[work]=evaluate_test_classes
    pred_test[work]=pred_test_classes
    df_submit=df_submit.append(df_base_classes, ignore_index=True)


df_submit.to_csv(doc_path +'/submit_fashion.csv')
df_submit.info()


for work in working_area:
    sub_classes = df_train[work].columns.tolist()[3:]
    for classes in sub_classes:
        plt.plot(hist_test[work][classes].history['loss'], color="#F7522F")
        plt.plot(hist_test[work][classes].history['val_loss'], color="#2F8FF7")
        plt.title('model loss of '+work+' '+classes)
        plt.ylabel('loss')
        plt.xlabel('epoch')
        plt.legend(['train','validation'], loc= 'upper left')
        plt.savefig(doc_path +'/'+work+'_'+classes+'_loss.png')
        

gc.collect()

Found 139467 images.
Found 24611 images.
Found 30135 images.
Epoch 1/10
1090/1090 [==============================] - 3873s 4s/step - loss: 1.2243 - acc: 0.6182 - val_loss: 1.4210 - val_acc: 0.5461
Epoch 2/10
1090/1090 [==============================] - 3834s 4s/step - loss: 1.0784 - acc: 0.6575 - val_loss: 1.4019 - val_acc: 0.5416
Epoch 3/10
1090/1090 [==============================] - 3812s 3s/step - loss: 1.0194 - acc: 0.6737 - val_loss: 1.5244 - val_acc: 0.5210
Epoch 4/10
1090/1090 [==============================] - 3785s 3s/step - loss: 0.9721 - acc: 0.6875 - val_loss: 1.6689 - val_acc: 0.5082
Epoch 5/10
1090/1090 [==============================] - 3790s 3s/step - loss: 0.9311 - acc: 0.6992 - val_loss: 1.7116 - val_acc: 0.5022
Epoch 6/10
1090/1090 [==============================] - 3793s 3s/step - loss: 0.8932 - acc: 0.7091 - val_loss: 1.5901 - val_acc: 0.5253
Epoch 7/10
1090/1090 [==============================] - 3782s 3s/step - loss: 0.8579 - acc: 0.7193 - val_loss: 1.9682 - val

Epoch 6/10
977/977 [==============================] - 3438s 4s/step - loss: 0.8759 - acc: 0.6807 - val_loss: 1.1730 - val_acc: 0.5436
Epoch 7/10
977/977 [==============================] - 3450s 4s/step - loss: 0.8388 - acc: 0.6956 - val_loss: 1.1310 - val_acc: 0.5529
Epoch 8/10
976/977 [============================>.] - ETA: 3s - loss: 0.7987 - acc: 0.7089